In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pandas import Series, DataFrame

In [ ]:
from google.colab import files
files.upload()

Saving SanDisk_Attrition_Original.csv to SanDisk_Attrition_Original (1).csv


{'SanDisk_Attrition_Original.csv': b'\xef\xbb\xbfAsic,Accounting/Finance,Age-Derived,All-Positions,Arizona State University,Avg. Tenure (before SanDisk),Job Hop,California State University,Computers,Degree Level,Degree Progam Rollup,Design,Director-VP-Mgmt,Engineer Type 1,Engineer Type,Engineer,Firmware,Gender,India,Indian Institute of Technology,Israel,Marketing/Sales,Number of Records,Other 1,Other 2,Other,Position level,Predict2,San Jose State University,SanDisk Position Change (from prior role),SanDisk Position,SanDisk Tenure,Santa Clara University,Software,Stanford University,Systems,Technion - Israel Institute of Technology,Total Experience,Total Other Job Experience,Total Other Jobs,Total SanDisk Jobs,Total SanDisk and Other Jobs,USA,USC,University Region,University of California,University of Illinois,University,Year,Id,Rf\r\n0,0,34,Computers,0,2.152328767,2-3,0,1,Bachelors,Engineering,0,0,India,Other,0,0,Men,1,0,0,0,1,1,0,0,IC,High,0,No Change,IT,3,0,0,0,0,0,14,10.76164384,5,1

In [ ]:
sdAttr = pd.read_csv("SanDisk_Attrition_Original.csv")

In [ ]:
sdAttr.shape

(1340, 51)

**Data Cleansing**

In [ ]:
# The following data cleaning steps can be run:

# 1) Remove duplicate rows from the given dataset
# 2) Round up the Risk Factor column to 2 decimal places\n
# 3) Drop duplicate rows again

In [ ]:
# Step 1: Drops the duplicate rows
sdAttr_1 = sdOrg.drop_duplicates()

In [ ]:
sdAttr_1.shape

(1232, 51)

In [ ]:
# Step 2a: Rounding up the Risk Factors to two decimal places
RefRf = sdAttr_1['Rf'].apply(lambda x: round(x, 2))

In [ ]:
# Step 2b: Adding a new column for the new Risk Factor
sdAttr_1['NewRf'] = RefRf

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
sdAttr_1.shape

(1232, 51)

In [ ]:
# Step 2c: Drop the 'Rf' column
sdAttr_1.drop(['Rf'], axis = 1, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Step 2d: Rename the column 'NewRf' to 'Rf'
sdAttr_1.rename(columns = {'NewRf': 'Rf'}, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Step 3: Obtain the final dataset after dropping duplicates again
sdAttrFinal = sdAttr_1.drop_duplicates()

In [ ]:
sdAttrFinal.shape

(1196, 51)

**Data Analysis**

In [ ]:
# Attrition is defined as the process of employees leaving the organization

# Attrition rate or percent is defined as the percentage of employees
# the organization lost and did not replace

In [ ]:
# Some analysis that could not be covered using Tableau are covered here.

# One of our primary objective is to find out the Attrition rate year-on-year.

In [ ]:
# Number of unique employees in each year
YrIdGp = sdAttrFinal.groupby("Year")["Id"].unique()

In [ ]:
# A series of employees year-wise
print(YrIdGp)

Year
2012    [181994, 185184, 189491, 11410, 186257, 181955...
2013    [11410, 185891, 186257, 185949, 186056, 181445...
2014    [185891, 189498, 11192, 185949, 186056, 181445...
Name: Id, dtype: object


In [ ]:
YearTotEmp = []
YearTotEmp.extend([len(YrIdGp[2012]), len(YrIdGp[2013]), len(YrIdGp[2014])])

In [ ]:
print(YearTotEmp)

[405, 374, 344]


In [ ]:
# Method 'setdiff1d' gives the elements in list1 that are not present in list2
AttrCountYear = [0]

AttrCountYear.append(len(np.setdiff1d(YrIdGp[2012], YrIdGp[2013])))
AttrCountYear.append(len(np.setdiff1d(YrIdGp[2013], YrIdGp[2014])))

In [ ]:
# The list contains the attrition count year-on-year
AttrCountYear

[0, 206, 192]

In [ ]:
# The number of original employees at the starting of the year
YrOrigEmp = []
YrOrigEmp.extend([YearTotEmp[0],
                  YearTotEmp[0] - AttrCountYear[1],
                  YearTotEmp[1] - AttrCountYear[2]])

In [ ]:
# List containing the number of employees at the starting of
# each year after deducting the attritions of the previous year.
YrOrigEmp

[405, 199, 182]

In [ ]:
# The number of hires in each of the years
YrHires = []
index = 0

while(index < 3):
  YrHires.append(YearTotEmp[index] - YrOrigEmp[index])
  index = index + 1

In [ ]:
# List containing the number of hires in each year
YrHires

[0, 175, 162]

In [ ]:
# Calculating the attrition list

# Formula for calculating the attrition rate = 
# ((Number of employees who left in the period) / (Average of the number of employees in the period)) * 100

AttrRateList = [0]
index = 1

while (index < 3):
  AttrRate = ((YearTotEmp[index - 1] - YearTotEmp[index]) /
              ((YearTotEmp[index - 1] + YearTotEmp[index]) / 2)) * 100
  AttrRateList.append(round(AttrRate, 2))
  index = index + 1


In [ ]:
# Attrition Rate year-on-year
AttrRateList

[0, 7.96, 8.36]

In [ ]:
# Creating a table from the data
AttrDf = DataFrame({'Orig': YrOrigEmp,
                    'Hires': YrHires,
                    'Total': YearTotEmp,
                    'Attr Rate': AttrRateList})

In [ ]:
# Attrition Table
AttrDf

,Orig,Hires,Total,Attr Rate
0,405,0,405,0.00
1,199,175,374,7.96
2,182,162,344,8.36
